In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

# import glob, os
# glob.glob(os.path.join(os.environ["SPARK_HOME"], "conf", "spark*"))
# print(os.environ)

In [2]:
#read csv

testdf = pd.read_csv('datasets/test.csv')
# traindf = pd.read_csv('datasets/train.csv')
alldf = pd.read_csv('datasets/submit.csv')

# print(testdf.shape, traindf.shape)

# print('cleaning dataframes')

# print(np.sum(testdf.isnull()))

testdf[['title', 'author']] = testdf[['title', 'author']].fillna(' ')

testdf = testdf.dropna()
# traindf = traindf.dropna()

# print(testdf.shape, traindf.shape)

ytestdf = testdf.drop(['title', 'author', 'text'], axis=1)

In [3]:
ytestdf = ytestdf.merge(alldf, left_on='id', right_on='id', how='left')[['id', 'label']]
# ytestdf = pd.merge(left = ytestdf, right=alldf, left_on='id', right_on='id', how='left')
ytest = ytestdf.label
print(ytest.shape)

(5193,)


In [4]:
spark = SparkSession.builder.\
    master('local[*]').\
        config('spark.driver.memory', '10g').\
            appName('fakenews').\
                getOrCreate()

print(spark.sparkContext.uiWebUrl)
print(spark.sparkContext.getConf().getAll())
    # master('local[*]').\
    #     config('spark.driver.memory','10g').\
print(spark)

http://ae652-7319.eait.uq.edu.au:4040
[('spark.app.name', 'fakenews'), ('spark.app.id', 'local-1648607599579'), ('spark.driver.memory', '10g'), ('spark.executor.id', 'driver'), ('spark.app.startTime', '1648607598569'), ('spark.sql.warehouse.dir', 'file:/C:/Users/s4355891/Documents/GitHub/fake-news-detection/spark-warehouse'), ('spark.driver.host', 'ae652-7319.eait.uq.edu.au'), ('spark.driver.port', '50917'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [5]:
from pyspark.sql.types import StructField, StructType
from pyspark.sql.types import IntegerType, StringType, DoubleType
from pyspark.sql import DataFrame
from pyspark.sql.functions import monotonically_increasing_id

trainschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType()),\
                StructField('label', IntegerType())]) 

testschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType())]) 

trainset = spark.read.csv('datasets/train.csv', header=True, schema=trainschema)
testset = spark.read.csv('datasets/test.csv', header=True, schema=testschema)
allset = spark.read.csv('datasets/submit.csv', header=True, inferSchema=True)


trainset = trainset.fillna(' ', subset=['title', 'author']).dropna(how='any')
testset = testset.fillna(' ', subset=['title', 'author']).dropna(how='any')

print(trainset.count(), testset.count())

# trainset.printSchema()
# testset.printSchema()

12256 5194


In [6]:
# testset.show(10)

testset = testset.join(allset, on='id', how='left')

In [7]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, CountVectorizer, StopWordsRemover, IDF
from pyspark.sql.functions import regexp_replace, udf

#remove punctuation and remove text with less than 10 words
def lower_clean_str(x):
    punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    lowercased_str = x.lower()
    for ch in punc:
        lowercased_str = lowercased_str.replace(ch,'')
    return lowercased_str

udfclean = udf(lower_clean_str, StringType())

trainset = trainset.withColumn('cleaned', udfclean(trainset.text))
testset = testset.withColumn('cleaned', udfclean(testset.text))

# regex = '[,\\-]'

# trainset = trainset.withColumn('text', regexp_replace(trainset.text, regex, ' '))
# testset = testset.withColumn('text', regexp_replace(testset.text, regex, ' '))

#tonkenize text

tokenizer = Tokenizer(inputCol='cleaned', outputCol='words')
# wordsdata = tokenizer.transform(trainset)

stopwr = StopWordsRemover(inputCol='words', outputCol='filtered')
# wordsdata = stopwr.transform(wordsdata)

hashing = HashingTF(inputCol='filtered', outputCol='hashed', numFeatures=32)
# wordsdata = hashing.transform(wordsdata)

hashingIDF = IDF(inputCol='hashed', outputCol='features')
# model = hashingIDF.fit(wordsdata)
# wordsdata = model.transform(wordsdata)

# vector = CountVectorizer(inputCol='filtered', outputCol='featuresCV')
# model = vector.fit(wordsdata)

# wordsdata = model.transform(wordsdata)
#create a model
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(maxIter=1)
# gbt.fit(wordsdata)

# wordsdata.show(10)

In [8]:
# testset.show(10)

In [9]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

pipeline = Pipeline(stages=[tokenizer, stopwr, hashing, hashingIDF, gbt])

pipeline = pipeline.fit(trainset.select(col('cleaned'), col('label').cast('int')))


predictions = pipeline.transform(testset.select(col('cleaned'))) 

Py4JJavaError: An error occurred while calling o170.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 8.0 failed 1 times, most recent failure: Lost task 9.0 in stage 8.0 (TID 32) (ae652-7319.eait.uq.edu.au executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$anonfun$map

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

Yspark_pred = predictions.toPandas()
Yspark_pred = Yspark_pred['prediction']

print(Yspark_pred.shape, testset.label.shape)

FK_accuracy = accuracy_score(ytestdf, Yspark_pred) * 100
FK_classification_report = classification_report(ytestdf, Yspark_pred)

print('The accuracy of GBT Classifier to predict fake news is ', FK_accuracy)
print('Classification Report: \n', FK_classification_report)

In [ ]:
spark.stop()